In [1]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
from qiskit.quantum_info import Statevector

qasm_filename = "qf21_n15_transpiled.qasm"
# Load the circuit from the QASM file
circuit = QuantumCircuit.from_qasm_file(qasm_filename)
circuit.remove_final_measurements()

circuit.save_statevector()

# Transpile for simulator
simulator = AerSimulator(method='statevector')
circuit = transpile(circuit, simulator)

# Run and get statevector
result = simulator.run(circuit).result()
statevector = result.get_statevector(circuit)
print(statevector)

Statevector([0.+0.j, 0.+0.j, 0.+0.j, ..., 0.+0.j, 0.+0.j, 0.+0.j],
            dims=(2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2))


In [ ]:
''''
from qiskit import QuantumCircuit
from qiskit.quantum_info import Statevector, Operator

# Load the circuit from the QASM file
circuit = QuantumCircuit.from_qasm_file(qasm_filename)
circuit.remove_final_measurements()

initial_state = Statevector.from_label('000000000000000')  # Start with |00⟩ state

# Step 3: Get the operator (unitary matrix) from the circuit
operator = Operator(circuit)

# Step 4: Evolve the initial state with the operator
final_state_evo = initial_state.evolve(operator)

# Step 5: Print the final statevector
print("Final Statevector:", final_state_evo)
''''

In [2]:
import csv
from qiskit.quantum_info import Operator

# Add this function to round near-zero values to exactly zero
def round_near_zero(matrix, threshold=1e-15):
    return [
        [
            complex(
                0 if abs(element.real) < threshold else element.real,
                0 if abs(element.imag) < threshold else element.imag
            )
            for element in row
        ]
        for row in matrix
    ]

# Load the circuit from the QASM file
circuit = QuantumCircuit.from_qasm_file(qasm_filename)
circuit.remove_final_measurements()

# Map qubits to their indices
qubit_indices = {qubit: idx for idx, qubit in enumerate(circuit.qubits)}

# Prepare data for CSV
output_data = []

# Extract the operational matrices of each gate along with qubit information
for i, instruction in enumerate(circuit.data):
    gate = instruction.operation  # The gate (operation)
    if gate.name != "measure" and gate.name !="barrier":
        qargs = instruction.qubits  # The qubits the gate acts on
        matrix = round_near_zero(Operator(gate).data) # Get the matrix representation
        
        # Collect gate information
        gate_info = [f"Gate {i + 1}", gate.name]
        if gate.num_qubits == 1:
            gate_info.append(f"") #empty control qubit
            gate_info.append(f"{qubit_indices[qargs[0]]}") #target only
        elif gate.num_qubits == 2:
            gate_info.append(f"{qubit_indices[qargs[0]]}") #control qubit
            gate_info.append(f"{qubit_indices[qargs[1]]}") #target qubit
        
        # Add matrix data row-wise
        gate_info.append(matrix)  # Convert matrix to list for CSV compatibility
        print(matrix)
        output_data.append(gate_info)

# Write the data to a CSV file
with open('quantum_circuit_gates.csv', mode='w', newline='') as file:
    writer = csv.writer(file)
    # Write headers
    writer.writerow(["Gate Number", "Gate Name", "Control Qubit", "Target Qubit","Matrix", circuit.num_qubits])
    # Write data rows
    writer.writerows(output_data)

print("Data written to quantum_circuit_gates.csv")

[[(0.7071067811865476-0.7071067811865476j), 0j], [0j, (0.7071067811865476+0.7071067811865476j)]]
[[(0.5+0.5j), (0.5-0.5j)], [(0.5-0.5j), (0.5+0.5j)]]
[[(0.7071067811865476-0.7071067811865476j), 0j], [0j, (0.7071067811865476+0.7071067811865476j)]]
[[(0.7071067811865476-0.7071067811865476j), 0j], [0j, (0.7071067811865476+0.7071067811865476j)]]
[[(0.5+0.5j), (0.5-0.5j)], [(0.5-0.5j), (0.5+0.5j)]]
[[(0.7071067811865476-0.7071067811865476j), 0j], [0j, (0.7071067811865476+0.7071067811865476j)]]
[[(0.7071067811865476-0.7071067811865476j), 0j], [0j, (0.7071067811865476+0.7071067811865476j)]]
[[(0.5+0.5j), (0.5-0.5j)], [(0.5-0.5j), (0.5+0.5j)]]
[[(0.7071067811865476-0.7071067811865476j), 0j], [0j, (0.7071067811865476+0.7071067811865476j)]]
[[(0.7071067811865476-0.7071067811865476j), 0j], [0j, (0.7071067811865476+0.7071067811865476j)]]
[[(0.5+0.5j), (0.5-0.5j)], [(0.5-0.5j), (0.5+0.5j)]]
[[(0.7071067811865476-0.7071067811865476j), 0j], [0j, (0.7071067811865476+0.7071067811865476j)]]
[[(0.7071067

In [ ]:
import numpy as np

def absolute_squared(state_vector):
    return np.abs(state_vector)**2

# Example usage
state_vector = np.array(statevector)
result = absolute_squared(state_vector)
print(result)

In [4]:
import numpy as np
import pandas as pd

# Load the gate information from CSV, ignoring extra columns
def load_gates_from_csv(file_path):
    data = pd.read_csv(file_path)
    # Drop the last column if there are extra columns beyond the expected
    if len(data.columns) > 5:  # Adjust based on the expected column count
        data = data.iloc[:, :-1]
    return data

# Convert the string representation of matrices to numpy arrays
def parse_matrix(matrix_str):
    return np.array(eval(matrix_str), dtype=complex)

# Apply a single-qubit gate using the given function
def apply_single_qubit_gate(state, gate, target_qubit, num_qubits):
    new_state = np.zeros_like(state)
    index_range = 1 << num_qubits  # 2^num_qubits

    # Calculate the correct indices and update only once per loop iteration
    for i in range(index_range):
        # Determine the bit value of the target qubit
        target_bit = (i >> target_qubit) & 1 #initial part to decide which state vector is affected by the operation
        # Find the index of the state where the target qubit is flipped
        flipped_i = i ^ (1 << target_qubit) 
        
        # Only update the new state vector at unique indices to avoid overwriting
        if i < flipped_i:  # This condition ensures each pair is handled once
            new_state[i] = gate[0, target_bit] * state[i] + gate[0, 1 - target_bit] * state[flipped_i]
            new_state[flipped_i] = gate[1, target_bit] * state[i] + gate[1, 1 - target_bit] * state[flipped_i]

    return new_state

# Apply a two-qubit gate using the given function
def apply_two_qubit_gate(state, gate, control_qubit, target_qubit, num_qubits):
    new_state = np.zeros_like(state, dtype=complex)
    index_range = 1 << num_qubits  # 2^num_qubits
    
    for i in range(index_range):
        # Determine the control and target bits
        control_bit = (i >> control_qubit) & 1
        target_bit = (i >> target_qubit) & 1

        # If control bit is 1, apply the gate on the target qubit
        if control_bit == 1:
            # Determine the index of the state where the target bit is flipped
            flipped_i = i ^ (1 << target_qubit)
            
            # Update the new state with the gate operation
            new_state[i] = gate[target_bit * 2 + control_bit, target_bit * 2 + control_bit] * state[i] + \
                           gate[(1 - target_bit) * 2 + control_bit, target_bit * 2 + control_bit] * state[flipped_i]
            new_state[flipped_i] = gate[target_bit * 2 + control_bit, (1 - target_bit) * 2 + control_bit] * state[i] + \
                                   gate[(1 - target_bit) * 2 + control_bit, (1 - target_bit) * 2 + control_bit] * state[flipped_i]
        else:
            # If control bit is 0, the state remains unchanged
            new_state[i] = state[i]

    return new_state

# Initialize the state vector (e.g., |000⟩ for 3 qubits)
def initialize_state_vector(num_qubits):
    state_vector = np.zeros(2**num_qubits, dtype=complex)
    state_vector[0] = 1  # Start in the |00...0⟩ state
    return state_vector

# Process gates and apply them to the state vector
def apply_gates_from_csv(file_path, num_qubits):
    gates_data = load_gates_from_csv(file_path)
    state_vector = initialize_state_vector(num_qubits)

    for _, row in gates_data.iterrows():
        gate_name = row['Gate Name']
        gate_matrix = np.array(parse_matrix(row['Matrix']))
        
        # Print gate name before application
        print(f"Applying Gate: {gate_name}")
        print(gate_matrix)
        print("\n")
        
        control_qubit = row.get('Control Qubit')
        target_qubit = row['Target Qubit']
        
        if pd.isna(control_qubit):  # Single-qubit gate
            state_vector = apply_single_qubit_gate(state_vector.copy(), gate_matrix, int(target_qubit), num_qubits)
        else:  # Two-qubit gate
            state_vector = apply_two_qubit_gate(state_vector.copy(), gate_matrix, int(control_qubit), int(target_qubit), num_qubits)
        #print(state_vector)
        print("\n")
    return state_vector

# Example usage:
file_path = 'quantum_circuit_gates.csv'
num_qubits = 15  # Adjust according to your system
final_state = apply_gates_from_csv(file_path, num_qubits)
print("Final State Vector:", final_state)

Applying Gate: rz
[[0.70710678-0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]




Applying Gate: sx
[[0.5+0.5j 0.5-0.5j]
 [0.5-0.5j 0.5+0.5j]]




Applying Gate: rz
[[0.70710678-0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]




Applying Gate: rz
[[0.70710678-0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]




Applying Gate: sx
[[0.5+0.5j 0.5-0.5j]
 [0.5-0.5j 0.5+0.5j]]




Applying Gate: rz
[[0.70710678-0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]




Applying Gate: rz
[[0.70710678-0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]




Applying Gate: sx
[[0.5+0.5j 0.5-0.5j]
 [0.5-0.5j 0.5+0.5j]]




Applying Gate: rz
[[0.70710678-0.70710678j 0.        +0.j        ]
 [0.        +0.j         0.70710678+0.70710678j]]




Applying Gate: rz
[[0.70710678-0.70710678j 0.        +0.j        ]
 [0.        

In [5]:
def compare_statevectors(vec1, vec2, tolerance=1e-6):
    """
    Compare two state vectors and identify any differences.
    
    Parameters:
    vec1 (np.ndarray): The first state vector.
    vec2 (np.ndarray): The second state vector.
    tolerance (float): The allowable tolerance for comparing the complex numbers.
    
    Returns:
    bool: True if the state vectors are identical within the given tolerance, False otherwise.
    List of differences if any are found.
    """
    # Ensure the input vectors are numpy arrays
    vec1 = np.array(vec1, dtype=np.complex128)
    vec2 = np.array(vec2, dtype=np.complex128)

    # Check if the lengths are equal
    if len(vec1) != len(vec2):
        print(f"State vectors have different lengths: {len(vec1)} vs {len(vec2)}")
        return False
    
    differences = []
    for i, (v1, v2) in enumerate(zip(vec1, vec2)):
        if not np.isclose(v1, v2, atol=tolerance):
            differences.append((i, v1, v2))
    
    if differences:
        print("Differences found:")
        for index, val1, val2 in differences:
            print(f"Index {index}: vec1 = {val1}, vec2 = {val2}")
        return False
    else:
        print("The state vectors are identical within the given tolerance.")
        return True

In [6]:
compare_statevectors(final_state, statevector)

The state vectors are identical within the given tolerance.


True

In [ ]:
compare_statevectors(final_state_evo, statevector)

In [ ]:
compare_statevectors(final_state_evo, final_state)

In [7]:

# Calculate probabilities and sort states from highest to lowest probability
probabilities = np.abs(statevector) ** 2

# Pair indices with their probabilities and sort by probability in descending order
state_probabilities = sorted(enumerate(probabilities), key=lambda x: x[1], reverse=True)

# Print the states and probabilities
for index, prob in state_probabilities:
    print(f"State {index}: {prob:.4f}")


State 22527: 0.0627
State 22015: 0.0444
State 22526: 0.0444
State 22014: 0.0317
State 21631: 0.0149
State 22520: 0.0149
State 22525: 0.0133
State 22271: 0.0133
State 22524: 0.0131
State 21759: 0.0131
State 22008: 0.0102
State 21630: 0.0102
State 22012: 0.0095
State 21758: 0.0095
State 22013: 0.0090
State 22270: 0.0090
State 22479: 0.0078
State 22431: 0.0073
State 22503: 0.0073
State 21823: 0.0062
State 22514: 0.0062
State 22430: 0.0058
State 21991: 0.0058
State 22496: 0.0053
State 21535: 0.0053
State 22335: 0.0052
State 22515: 0.0052
State 21967: 0.0050
State 22478: 0.0050
State 22516: 0.0049
State 21695: 0.0049
State 22512: 0.0048
State 21567: 0.0048
State 22334: 0.0043
State 22003: 0.0043
State 22002: 0.0042
State 21822: 0.0042
State 21887: 0.0041
State 22522: 0.0041
State 22513: 0.0041
State 22079: 0.0041
State 21919: 0.0040
State 22502: 0.0040
State 22523: 0.0036
State 22399: 0.0036
State 21504: 0.0034
State 21984: 0.0033
State 21534: 0.0033
State 22004: 0.0033
State 21694: 0.0033
